# Basic Multi-Agent Example with AutoGen

This notebook implements a simple multi-agent system using AutoGen. 

In [ ]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
import timeit
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
import sys

sys.path.append(os.path.abspath(".."))
from utilities import (
    create_chat_completion_client,    
    adjust_properties_for_fixed_response,    
    load_model_config,
)

#### Note: You need a tavily API key to use the web search tool!

In [ ]:
# model = load_model_config()
model = load_model_config(model_name="gpt-oss:20b")
# model = load_model_config(model_name="qwen3")

Using model: qwen3 type: ollama


Set the joke's topic
----------------------

In [3]:
topic = "Animals"

Select Model and Agent Configuration
-----------------------------

In [4]:
agentConfig = {
    "Author": dict(
        system_message="Write a SHORT joke above the provided topic."
    ),
    "SmartassEditor": dict(
        system_message="Analyze the joke, explain why it is funny or not funny, and make it funnier without making it longer.",
    ),
}

# For testing, we can fix an agent's response to a known answer
#adjust_properties_for_fixed_response(agentConfig["Author"], "What do you call a magic dog? A Labracadabrador.")

In [5]:
# Now create model client
model_client = create_chat_completion_client(model)

agents = []  # start with a user proxy for the researcher

for agent_type, props in agentConfig.items():
    agents.append(
        AssistantAgent(
            name=agent_type,
            **props,  # type: ignore
            model_client=model_client,
        )
    )

model_config {'name': 'qwen3', 'type': 'ollama', 'host': 'https://ollama.cloud.intern.mevis.fraunhofer.de'}


In [6]:
# Set up termination conditions
termination = MaxMessageTermination(max_messages=10) #len(agents))

# Create the team
team = RoundRobinGroupChat(
    participants=agents,
    termination_condition=termination,
    max_turns=len(agents) # we don't really want it to go round
)


Start the System
===================

In [7]:
async def invent_joke(topic: str):
    start_time_s = timeit.default_timer()
    result = await team.run(task=f"Invent a short joke about: {topic}")
    elapsed = timeit.default_timer()-start_time_s
    print(f"🔍 Generation result (took {elapsed:.2f} sec.) resulted in {len(result.messages)} messages.")
    return result  # Return the result for further exploration

# Run the deep dive
result = await invent_joke(topic)

Error processing publish message for Author_5a59898f-9289-4e29-bcbf-729095be1acc/5a59898f-9289-4e29-bcbf-729095be1acc
Traceback (most recent call last):
  File "c:\dev\git\autogen_experiments\.venv\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
    )
    ^
  File "c:\dev\git\autogen_experiments\.venv\Lib\site-packages\autogen_core\_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\dev\git\autogen_experiments\.venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\dev\git\autogen_experiments\.venv\Lib\site-packages\autogen_core\_routed_agent.py", line 485, in on_message_i

RuntimeError: ResponseError: model 'qwen3' not found (status code: 404)
Traceback:
Traceback (most recent call last):

  File "c:\dev\git\autogen_experiments\.venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_chat_agent_container.py", line 133, in handle_request
    async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):
    ...<4 lines>...
            await self._log_message(msg)

  File "c:\dev\git\autogen_experiments\.venv\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 953, in on_messages_stream
    async for inference_output in self._call_llm(
    ...<15 lines>...
            yield inference_output

  File "c:\dev\git\autogen_experiments\.venv\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 1107, in _call_llm
    model_result = await model_client.create(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<4 lines>...
    )
    ^

  File "c:\dev\git\autogen_experiments\.venv\Lib\site-pack

Explore Results
===============

In [ ]:
# Display the full result structure
print("Result type:", type(result))

# print("\nResult attributes:")
# print(get_attributes(result))
# print("\nMessage attributes:")
# print(get_attributes(result.messages[-1]))

print("Message Sequence:")
for i, m in enumerate(result.messages):
    print(f"\n----------------------------------------------------------------")
    print(f"MESSAGE {i} [{m.source}]:\n{m.content}") # type: ignore
    

In [ ]:
# Clean up resources
await model_client.close()

In [ ]:
import re
from autogen_agentchat.messages import BaseTextChatMessage

print("Stop reason:", result.stop_reason)
assert isinstance(result.messages[-1], BaseTextChatMessage)

print("Message Sequence:")
for m in result.messages:
    assert isinstance(m, BaseTextChatMessage), f"Unexpected message type: {type(m)}"
    message = m.content
    print(f" - {m.source}: {message}")
